# Latest dataset exploration - n=131
This notebook focuses on the implementation of the Cob class, which encompasses the analysis developed as part of the initial exploration of 15min sampled data as the output of Isabella Dagen's insulin-need project preprocessing. It uses the Cob class from cob_analysis.py to analyse the data and create peak features to investigate the existence of diurnal patterns in the data.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from src.cob_analysis import Cob



2025-05-11 21:40:57.817 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Users\ross\OneDrive\Documents\Masters\Project\masters_project


In [12]:
# Previous preprocessing - 15 min sampled data based on earlier limited dataset
cob = Cob()
cob.read_interim_data(file_name='15min_iob_cob_bg',
                      file_type='csv',
                      sampling_rate=15)

Number of records: 599360
Number of people: 116
Systems used: ['OpenAPS']
Categories (1, object): ['OpenAPS']
